# Part One

You can hear birds chirping and raindrops hitting leaves as the expedition proceeds. <br>Occasionally, you can even hear much louder sounds in the distance; how big do the animals get out here, anyway?

The device the Elves gave you has problems with more than just its communication system. You try to run a system update:

$ system-update --please --pretty-please-with-sugar-on-top
Error: No space left on device
Perhaps you can delete some files to make space for the update?

You browse around the filesystem to assess the situation and save the resulting terminal output (your puzzle input). For example:

$ cd / <br>
$ ls <br>
dir a <br>
14848514 b.txt <br>
8504156 c.dat <br>
dir d  <br>
$ cd a  <br>
$ ls  <br>
dir e<br>
29116 f<br>
2557 g<br>
62596 h.lst<br>
$ cd e<br>
$ ls<br>
584 i<br>
$ cd ..<br>
$ cd ..<br>
$ cd d<br>
$ ls<br>
4060174 j<br>
8033020 d.log<br>
5626152 d.ext<br>
7214296 k<br>
The filesystem consists of a tree of files (plain data) and directories (which can contain other directories or files). <br>The outermost directory is called /. You can navigate around the filesystem, moving into or out of directories and listing the contents of the directory you're currently in.

Within the terminal output, lines that begin with $ are commands you executed, very much like some modern computers:

cd means change directory. This changes which directory is the current directory, but the specific result depends on the argument:<br>
cd x moves in one level: it looks in the current directory for the directory named x and makes it the current directory.<br>
cd .. moves out one level: it finds the directory that contains the current directory, then makes that directory the current directory.<br>
cd / switches the current directory to the outermost directory, /.<br>
ls means list. It prints out all of the files and directories immediately contained by the current directory:<br>
123 abc means that the current directory contains a file named abc with size 123.<br>
dir xyz means that the current directory contains a directory named xyz.<br>
Given the commands and output in the example above, you can determine that the filesystem looks visually like this:<br>

- / (dir)
  - a (dir)
    - e (dir)
      - i (file, size=584)
    - f (file, size=29116)
    - g (file, size=2557)
    - h.lst (file, size=62596)
  - b.txt (file, size=14848514)
  - c.dat (file, size=8504156)
  - d (dir)
    - j (file, size=4060174)
    - d.log (file, size=8033020)
    - d.ext (file, size=5626152)
    - k (file, size=7214296)
Here, there are four directories: / (the outermost directory), a and d (which are in /), and e (which is in a). <br>These directories also contain files of various sizes.

Since the disk is full, your first step should probably be to find directories that are good candidates for deletion. <br>To do this, you need to determine the total size of each directory. The total size of a directory is the sum of the sizes of the files it contains, directly or indirectly. (Directories themselves do not count as having any intrinsic size.)

The total sizes of the directories above can be found as follows:

The total size of directory e is 584 because it contains a single file i of size 584 and no other directories.
The directory a has total size 94853 because it contains files f (size 29116), g (size 2557), and h.lst (size 62596), <br>plus file i indirectly (a contains e which contains i).
Directory d has total size 24933642.
As the outermost directory, / contains every file. Its total size is 48381165, the sum of the size of every file.
To begin, find all of the directories with a total size of at most 100000, then calculate the sum of their total sizes. <br>In the example above, these directories are a and e; the sum of their total sizes is 95437 (94853 + 584). (As in this example, this process can count files more than once!)

Find all of the directories with a total size of at most 100000. What is the sum of the total sizes of those directories?

In [308]:
from string import digits
from functools import reduce
from collections import defaultdict

In [206]:
path = "../data/puzzle_7.txt"
# path = "../data/sample.txt"

In [207]:
with open(path, 'r') as file:
  # Read the contents of the file into a list
  data = file.readlines()

In [208]:
data = [x.replace("\n", "") for x in data]

## Some useful functions to extract and set values in a nested dictionary 

e.g. given the dictionary {"a": {"b": {"c": 1}}} and the string "a/b/c" - I can get the value of 1 or change the value

In [209]:
def get_value(d, path):
    return reduce(lambda d, k: d[k], path.split('/'), d)

In [210]:
def set_value(d, path, value):
    keys = path.split('/')
    current_d = d
    for key in keys[:-1]:
        current_d = current_d.setdefault(key, {})
    current_d[keys[-1]] = value

## Instead I used a defaultdict and iterated over the current_path to add the file size to all directories above the current

In [301]:
store = defaultdict(int)

In [302]:
current_path = []
for i in data:
    if "$ cd " in i and i != "$ cd ..":
        current_dir = i.split(" ")[-1]
        current_dir = "~" if current_dir == "/" else current_dir
        current_path.append(current_dir)
    elif i == "$ cd ..":
        _ = current_path.pop()
    elif any([n in i.split(" ")[0] for n in digits]):
        # Iterate through every dir in the full path to the file
        for j in range(len(current_path)):
            dir = '/'.join(current_path[:j+1]).replace("//", "/")
            store[dir] += int(i.split(" ")[0])

In [307]:
sum([x for x in store.values() if x <= 100000])

1350966

# Part Two

Find the smallest directory that, if deleted, would free up enough space on the filesystem to run the update. What is the total size of that directory?

In [312]:
total_space = 70000000
required_space = 30000000
used_space = store["~"]

In [317]:
used_space + required_space - total_space

6233734

In [321]:
min([v for k,v in store.items() if v >= used_space + required_space - total_space])

6296435